# getter와 setter 메서드 대신에 일반 속성을 사용하자

다른 언어에서 파이썬으로 넘어온 프로그래머들은 자연스레 클래스에서 getter와 setter 메서드를 명시적으로 구현하려 한다.

In [1]:
class OldResistor(object):
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms

getter와 setter를 사용하는 방법은 간단하지만 파이썬답지 않다.

In [2]:
r0 = OldResistor(50e3)
print('Before: %5r' % r0.get_ohms())
r0.set_ohms(10e3)
print('After:  %5r' % r0.get_ohms())

Before: 50000.0
After:  10000.0


getter와 setter 메서드는 특히 즉석에서 증가시키기 같은 연산에는 사용하기 불편하다.

In [3]:
r0.set_ohms(r0.get_ohms() + 5e3)

이런 유틸리티 메서드는 클래스의 인터페이스를 정의하는 데 도움이 되고 기능을 캡슐화하고 사용법을 검증하고 경계를 정의하기 쉽게 해준다. 이런 요소는 클래스가 시간이 지나면서 발전하더라도 호출하는 쪽 코드를 절대 망가뜨리지 않도록 설계할 때 중요한 목표가 된다.

하지만 파이썬에서는 명시적인 getter와 setter를 구현할 일이 없다. 대신 간단한 공개 속성부터 구현해야한다.

In [4]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
r1 = Resistor(50e3)
r1.ohms = 10e3

즉석에서 증가시키기 같은 연산이 자연스럽고 명확해진다.

In [5]:
r1.ohms += 5e3

나중에 속성을 설정할 때 특별한 동작이 일어나야 하면 `@property` 데코레이터와 이에 대응하는 setter 속성을 사용하는 방법으로 바꿀 수 있다. 여기서는 `Resistor`의 새 서브클래스를 정의하여 `voltage` property를 할당하면 current 값이 바뀌게 해본다. 제대로 동작하려면 setter와 getter 메서드의 이름이 의도한 property 이름과 일치해야 한다.

In [7]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

이제 voltage property에 할당하면 voltage setter method가 실행되어 voltage에 맞게 객체의 current property를 업데이트할 것이다.

In [8]:
r2 = VoltageResistance(1e3)
print('Before: %5r amps' % r2.current)
r2.voltage = 10
print('After:  %5r amps' % r2.current)

Before:     0 amps
After:   0.01 amps


프로퍼티에 setter를 설정하면 클래스에 전달된 값들의 타입을 체크하고 값을 검증할 수도 있다. 다음은 모든 저항값이 0옴보다 큼을 보장하는 클래스를 정의한 것이다.

In [9]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
        self._ohms = ohms

In [10]:
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: 0.000000 ohms must be > 0

In [11]:
BoundedResistance(-5)

ValueError: -5.000000 ohms must be > 0

이 예외는 `BoundedResistance.__init__`가 `self.ohms = -5`를 할당하는 `Resistor.__init__`를 호출하기 때문에 일어난다. 이 할당문으로 `BoundedResistance`의 `@ohms.setter` 메서드가 호출되어 객체 생성이 완료되기도 전에 곧장 검증 코드가 실행된다.

부모 클래스의 속성을 immutable으로 만드는 데도 `@property`를 사용할 수 있다.

In [12]:
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms

In [14]:
r4 = FixedResistance(1e3)
r4.ohms = 2e3

AttributeError: Can't set attribute

`@property`의 가장 큰 단점은 속성에 대응하는 메서드를 서브클래스에서만 공유할 수 있다는 점이다. 서로 관련이 없는 클래스는 같은 구현을 공유하지 못한다. 하지만 파이썬은 재사용 가능한 프로퍼티 로직을 비롯해 다른 많은 쓰임새를 가능하게 하는 descriptor도 지원한다.

마지막으로 `@property` 메서드로 setter와 getter를 구현할 때 예상과 다르게 동작하지 않게 해야한다.

예를 들면 getter 프로퍼티 메서드에서 다른 속성을 설정하지 말아야 한다.

In [15]:
class MysteriousResistor(Resistor):
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms
        
r7 = MysteriousResistor(10)
r7.current = 0.01
print('Before: %5r' % r7.voltage)
r7.ohms
print('After:  %5r' % r7.voltage)

Before:     0
After:    0.1


최선의 정책은 `@property.setter` 메서드에서만 관련 객체의 상태를 수정하는 것이다. 모듈을 동적으로 임포트하거나, 느린 핼퍼 함수를 실행하거나, 비용이 많이 드는 데이터베이스 쿼리를 수행하는 일처럼 호출하는 쪽이 객체에서 일어날 것이라고 예측하지 못할 만한 다른 부작용은 모두 피해야 한다.